In [1]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from loguru import logger
from pathlib import Path
import numpy as np
import spatialdata as sd
from napari_spatialdata import Interactive

from plex_pipe.utils.config_loaders import load_analysis_settings
from plex_pipe.processors import build_processor
from plex_pipe.processors.controller import ResourceBuildingController

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\scanpy\_utils\__init__.py:33: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  from

### Load analysis settings

In [2]:
# load analysis configuration
settings_path = r'C:\BLCA\BLCA-4_Analysis\analysis_BLCA4.yaml'

overwrite_mask = True

settings = load_analysis_settings(settings_path)

2025-11-19 11:30:49.016 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:51 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


### Define the logger

In [3]:
log_file = settings.log_dir_path / f"cores_segmenation_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

### Define cores for the analysis

In [4]:
core_dir = settings.analysis_dir / 'cores'
path_list = [core_dir / f for f in os.listdir(core_dir)]
path_list.sort()
path_list

[WindowsPath('C:/BLCA/BLCA-4_Analysis/cores/Core_000.zarr')]

### Setup

In [5]:
# setup builders of additional data elements

if getattr(settings,'additional_elements',None):
    
    builders_list = []

    for builder_settings in settings.additional_elements:
        
        params = dict(getattr(builder_settings,'parameters',None)) or {}

        builder = build_processor(builder_settings.category, builder_settings.type, **params) 
        
        builder_controller = ResourceBuildingController(builder=builder, 
                                            input_names=builder_settings.input, 
                                            output_names=builder_settings.output, 
                                            keep=builder_settings.keep, 
                                            overwrite=True,
                                            pyramid_levels=settings.sdata_storage.max_pyramid_level,
                                            downscale = settings.sdata_storage.downscale,
                                            chunk_size = settings.sdata_storage.chunk_size,
                                            )
        
        logger.info(f"Image transformer of type '{builder_settings.type}' for image '{builder_settings.input}' has been created.")

        builders_list.append(builder_controller)

else:
    builders_list = []
    logger.info("No resource builders specified.")

2025-11-19 11:30:57.328 | INFO     | __main__:<module>:23 - Image transformer of type 'normalize' for image 'DAPI' has been created.
2025-11-19 11:30:57.329 | INFO     | __main__:<module>:23 - Image transformer of type 'normalize' for image 'CD45' has been created.
2025-11-19 11:30:57.329 | INFO     | __main__:<module>:23 - Image transformer of type 'normalize' for image 'CD44' has been created.
2025-11-19 11:30:57.329 | INFO     | __main__:<module>:23 - Image transformer of type 'normalize' for image 'HLA1' has been created.
2025-11-19 11:30:57.330 | INFO     | __main__:<module>:23 - Image transformer of type 'normalize' for image 'NaKATPase' has been created.
2025-11-19 11:30:57.330 | INFO     | __main__:<module>:23 - Image transformer of type 'normalize' for image 'CD11C' has been created.
2025-11-19 11:30:57.330 | INFO     | __main__:<module>:23 - Image transformer of type 'normalize' for image 'pCK26' has been created.
2025-11-19 11:30:57.330 | INFO     | __main__:<module>:23 - Im

### Processing

In [9]:
# Optional - to detect problems early (runs <10s per sdata object)
for sd_path in path_list:
    
    logger.info(f"Validating {sd_path.name}")

    # get sdata
    sdata = sd.read_zarr(sd_path)

    # check that the pipeline can run on provide sdata
    settings.validate_pipeline(sdata)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-19 10:45:04.515 | INFO     | __main__:<module>:4 - Validating Core_000.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mi

2025-11-19 10:45:10.784 | INFO     | plex_pipe.utils.config_schema:validate_pipeline:188 - ✅ Pipeline validation successful.


In [6]:
for sd_path in path_list:
    
    logger.info(f"Processing {sd_path.name}")

    # get sdata
    sdata = sd.read_zarr(sd_path)

    # check that the pipeline can run on provide sdata
    settings.validate_pipeline(sdata)

    # run builders of additional elements
    for builder_controller in builders_list:
        sdata = builder_controller.run(sdata)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-19 11:31:03.347 | INFO     | __main__:<module>:3 - Processing Core_000.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mi

2025-11-19 11:31:05.947 | INFO     | plex_pipe.utils.config_schema:validate_pipeline:188 - ✅ Pipeline validation successful.
2025-11-19 11:31:05.948 | INFO     | plex_pipe.processors.controller:validate_resolution_present:85 - All channels have required resolution level: 0
2025-11-19 11:31:07.064 | INFO     | plex_pipe.processors.image_filters:run:73 - Applied normalization (percentiles 1.0–99.8) → [98.0, 2809.0]
2025-11-19 11:31:07.068 | INFO     | plex_pipe.processors.controller:run:212 - New element(s) '['DAPI_norm']' have been created.
2025-11-19 11:31:07.249 | INFO     | plex_pipe.processors.controller:validate_resolution_present:85 - All channels have required resolution level: 0
2025-11-19 11:31:08.076 | INFO     | plex_pipe.processors.image_filters:run:73 - Applied normalization (percentiles 1.0–99.8) → [0.0, 512.0]
2025-11-19 11:31:08.080 | INFO     | plex_pipe.processors.controller:run:212 - New element(s) '['CD45_norm']' have been created.
2025-11-19 11:31:08.198 | INFO     

D:\instanseg\instanseg\utils\pytorch_utils.py:297: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  intersection = torch.sparse.mm(onehot1, onehot2.T).to_dense()


2025-11-19 11:31:39.740 | INFO     | plex_pipe.processors.controller:run:212 - New element(s) '['instanseg_nucleus', 'instanseg_cell']' have been created.
2025-11-19 11:31:41.116 | INFO     | plex_pipe.processors.controller:validate_resolution_present:85 - All channels have required resolution level: 0
2025-11-19 11:31:41.116 | WARNING  | plex_pipe.processors.controller:prepare_to_overwrite:112 - Mask name 'ring' already exists and will be overwritten.
2025-11-19 11:31:41.116 | INFO     | plex_pipe.processors.controller:prepare_to_overwrite:116 - Existing element 'ring' deleted from sdata.
2025-11-19 11:31:42.495 | INFO     | plex_pipe.processors.controller:prepare_to_overwrite:119 - Existing element 'ring' deleted from disk.
2025-11-19 11:31:48.659 | INFO     | plex_pipe.processors.controller:run:212 - New element(s) '['ring']' have been created.
2025-11-19 11:31:49.756 | INFO     | plex_pipe.processors.controller:run:233 - Mask 'ring' has been saved to disk.
2025-11-19 11:31:49.766 |

### Sneak peek

In [7]:
# refresh the object
sdata_org = sd.read_zarr(path_list[0])
sdata_org

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requeste

SpatialData object, with associated Zarr store: C:\BLCA\BLCA-4_Analysis\cores\Core_000.zarr
├── Images
│     ├── '53BP1': DataTree[cyx] (1, 5056, 5376), (1, 2528, 2688), (1, 1264, 1344)
│     ├── 'CD3': DataTree[cyx] (1, 5056, 5376), (1, 2528, 2688), (1, 1264, 1344)
│     ├── 'CD8a': DataTree[cyx] (1, 5056, 5376), (1, 2528, 2688), (1, 1264, 1344)
│     ├── 'CD11C': DataTree[cyx] (1, 5056, 5376), (1, 2528, 2688), (1, 1264, 1344)
│     ├── 'CD11b': DataTree[cyx] (1, 5056, 5376), (1, 2528, 2688), (1, 1264, 1344)
│     ├── 'CD20': DataTree[cyx] (1, 5056, 5376), (1, 2528, 2688), (1, 1264, 1344)
│     ├── 'CD31': DataTree[cyx] (1, 5056, 5376), (1, 2528, 2688), (1, 1264, 1344)
│     ├── 'CD44': DataTree[cyx] (1, 5056, 5376), (1, 2528, 2688), (1, 1264, 1344)
│     ├── 'CD45': DataTree[cyx] (1, 5056, 5376), (1, 2528, 2688), (1, 1264, 1344)
│     ├── 'CD45RO': DataTree[cyx] (1, 5056, 5376), (1, 2528, 2688), (1, 1264, 1344)
│     ├── 'CD68': DataTree[cyx] (1, 5056, 5376), (1, 2528, 2688), (1, 126

In [ ]:
Interactive(sdata)

2025-11-19 11:34:17.693 | WARNING  | napari_spatialdata._viewer:__init__:57 - Due to Shift-L being used as shortcut in napari, it is being deprecated and might not link a new layer to an existing SpatialData object in the viewer. Please use ⌘-L on MacOS or else Ctrl-L.


2025-11-19 11:35:14.971 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-19 11:35:15.275 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.


In [10]:
sdata_org

SpatialData object, with associated Zarr store: C:\sdata_todel\LCB004-N-P_Analysis\cores\Core_000.zarr
├── Images
│     ├── 'AP2B': DataTree[cyx] (1, 576, 640), (1, 288, 320), (1, 144, 160)
│     ├── 'AR': DataTree[cyx] (1, 576, 640), (1, 288, 320), (1, 144, 160)
│     ├── 'CD45': DataTree[cyx] (1, 576, 640), (1, 288, 320), (1, 144, 160)
│     ├── 'CK14': DataTree[cyx] (1, 576, 640), (1, 288, 320), (1, 144, 160)
│     ├── 'CK818': DataTree[cyx] (1, 576, 640), (1, 288, 320), (1, 144, 160)
│     ├── 'DAPI': DataTree[cyx] (1, 576, 640), (1, 288, 320), (1, 144, 160)
│     ├── 'ECad': DataTree[cyx] (1, 576, 640), (1, 288, 320), (1, 144, 160)
│     ├── 'ER': DataTree[cyx] (1, 576, 640), (1, 288, 320), (1, 144, 160)
│     ├── 'FOXA1': DataTree[cyx] (1, 576, 640), (1, 288, 320), (1, 144, 160)
│     ├── 'GATA3': DataTree[cyx] (1, 576, 640), (1, 288, 320), (1, 144, 160)
│     ├── 'HER2': DataTree[cyx] (1, 576, 640), (1, 288, 320), (1, 144, 160)
│     ├── 'ProgRc': DataTree[cyx] (1, 576, 640), (1

In [13]:
getattr(sdata_org, "tables", {})

{'instanseg_table': AnnData object with n_obs × n_vars = 386 × 105
    obs: 'label', 'area_nucleus', 'eccentricity_nucleus', 'solidity_nucleus', 'perimeter_nucleus', 'euler_number_nucleus', 'area_cell', 'eccentricity_cell', 'solidity_cell', 'perimeter_cell', 'euler_number_cell', 'area_ring', 'eccentricity_ring', 'solidity_ring', 'perimeter_ring', 'euler_number_ring', 'area_cyto', 'eccentricity_cyto', 'solidity_cyto', 'perimeter_cyto', 'euler_number_cyto', 'region', 'cell'
    uns: 'spatialdata', 'spatialdata_attrs'
    obsm: 'centroid_cell', 'centroid_cyto', 'centroid_nucleus', 'centroid_ring'
    layers: 'qc_mask'}